In [ ]:
import sys
sys.stdout.reconfigure(encoding='utf-8')

def validate_float(prompt, min_value=0, max_value=float('inf')):
    while True:
        try:
            value = float(input(prompt))
            if min_value <= value <= max_value:
                return value
            else:
                print(f"Please enter a value between {min_value} and {max_value}.")
        except ValueError:
            print("Please enter a valid number.")

def validate_choice(prompt, choices):
    choice_map = {str(i+1): c for i, c in enumerate(choices)}
    while True:
        print(prompt)
        for k, v in choice_map.items():
            print(f"{k}. {v}")
        selection = input("Enter choice number: ")
        if selection in choice_map:
            return choice_map[selection]
        else:
            print("Invalid choice. Please try again.")

def choose_unit_system():
    choice = input("Choose your unit system: [1] Imperial (miles, gallons), [2] Metric (km, liters): ").strip()
    return 'metric' if choice == '2' else 'imperial'

def transportation(unit_system):
    print("\n--- Transportation ---")
    if unit_system == 'imperial':
        miles = validate_float("Miles driven per week: ")
        mpg = validate_float("Fuel efficiency (mpg): ", 5, 100)
        annual_miles = miles * 52
        emissions = (annual_miles / mpg) * 8.89 / 1000
        return emissions, f"{annual_miles:.0f} miles/year"
    else:
        km = validate_float("Kilometers driven per week: ")
        l_per_100km = validate_float("Fuel efficiency (L/100km): ", 2, 30)
        liters_per_km = l_per_100km / 100
        annual_km = km * 52
        emissions = annual_km * liters_per_km * 2.31 / 1000
        return emissions, f"{annual_km:.0f} km/year"

def energy():
    print("\n--- Home Energy ---")
    monthly_kwh = validate_float("Average monthly electricity use (kWh): ", 0)
    annual_kwh = monthly_kwh * 12
    elec_emission_factor = 0.92e-3
    return annual_kwh * elec_emission_factor, f"{annual_kwh:.0f} kWh/year"

def diet():
    print("\n--- Diet ---")
    diet_type = validate_choice("Select your diet type:", ["Vegan", "Vegetarian", "Pescatarian", "Omnivore"])
    emission_factors = {
        "Vegan": 1.5,
        "Vegetarian": 1.7,
        "Pescatarian": 1.9,
        "Omnivore": 2.5
    }
    return emission_factors[diet_type], diet_type

def waste():
    print("\n--- Waste ---")
    recycling = validate_choice("Do you recycle regularly?", ["Yes", "No"])
    waste_emission = 0.25 if recycling == "Yes" else 0.6
    return waste_emission, recycling

def summarize(emissions, highlight_threshold=0.25):
    total = sum(val[0] for val in emissions.values())
    print("\n===== Annual Carbon Footprint Summary =====")
    for category, (value, detail) in emissions.items():
        percent = value / total
        marker = "\U0001F53A" if percent >= highlight_threshold else "   "
        print(f"{marker} {category}: {value:.2f} metric tons CO2e ({detail})")
    print(f"\nTOTAL: {total:.2f} metric tons CO2e")
    print(f"\U0001F30D Equivalent to driving {total * 2500:.0f} miles or powering {total / 4.6:.1f} homes/year.")
    print(f"\U0001F4CA Compared to U.S. Average (16.5 tons): {total / 16.5 * 100:.1f}%")
    earths = total / 1.7
    print(f"\U0001F30E If everyone lived like you, we’d need {earths:.1f} Earths.")
    return total

def export_summary(emissions, total):
    filename = input("Enter a filename to save your report (e.g., my_footprint.txt): ").strip()
    if not filename:
        filename = "carbon_footprint_report.txt"
    with open(filename, 'w') as f:
        f.write("Carbon Footprint Report\n")
        f.write("=======================\n")
        for category, (value, detail) in emissions.items():
            f.write(f"{category}: {value:.2f} metric tons CO2e ({detail})\n")
        f.write(f"\nTOTAL: {total:.2f} metric tons CO2e\n")
        f.write("U.S. Average: 16.5 tCO2e\n")
        f.write(f"You are at {total / 16.5 * 100:.1f}% of the U.S. average.\n")
    print(f"\u2705 Report saved to {filename}")

def recommend(emissions, threshold=0.25):
    total = sum(val[0] for val in emissions.values())
    print(f"\n\U0001F4CC Emission Breakdown (threshold: {threshold*100:.0f}%)")
    suggestions = {
        "Transportation": "\U0001F697 Consider carpooling, using public transport, or switching to an EV.",
        "Energy": "\u26A1 Switch to LED bulbs, unplug devices, or explore renewable energy.",
        "Diet": "\U0001F957 Try reducing meat intake or adding plant-based meals.",
        "Waste": "\u267B️ Recycle regularly and reduce single-use plastics."
    }
    flagged = False
    for category, (value, _) in emissions.items():
        percent = value / total
        if percent >= threshold:
            print(f"- {category}: {value:.2f} tons ({percent*100:.1f}%) — {suggestions[category]}")
            flagged = True
    if not flagged:
        print("\u2705 All categories are below the threshold. Keep up the good habits!")

def show_rank(total):
    print("\n===== Performance Ranking =====")
    avg_us = 16.5
    delta = avg_us - total
    if total >= avg_us:
        print("\u274C You are above the U.S. average. Try the suggestions to reduce your impact.")
    elif total > 12:
        print("\ud83e\udd49 Bronze Rank — You're doing better than many Americans.")
    elif total > 8:
        print("\ud83e\udd48 Silver Rank — Great work reducing your footprint!")
    else:
        print("\ud83e\udd47 Gold Rank — You're ahead of the curve! Keep leading by example.")
    print(f"\U0001F4C9 You're emitting {delta:.2f} tons less than the average American.")

def apply_suggestions(emissions):
    print("\n\u2705 Applying smart suggestions to simulate reduced emissions...")
    if emissions["Transportation"][0] > 0:
        emissions["Transportation"] = (emissions["Transportation"][0] * 0.7, emissions["Transportation"][1])
    if emissions["Energy"][0] > 0:
        emissions["Energy"] = (emissions["Energy"][0] * 0.8, emissions["Energy"][1])
    if emissions["Diet"][1] != "Vegan":
        emissions["Diet"] = (1.5, "Improved (Vegan)")
    if emissions["Waste"][1] == "No":
        emissions["Waste"] = (0.25, "Now Recycling")
    print("\U0001F389 Reductions applied! Let's see your updated footprint...")

def test_manual_changes(emissions):
    print("\n\U0001F9EA Manual What-If Mode")
    category_map = {
        "1": ("Transportation", transportation),
        "2": ("Energy", energy),
        "3": ("Diet", diet),
        "4": ("Waste", waste)
    }
    while True:
        print("\nWhich category do you want to test?")
        for k, (name, _) in category_map.items():
            print(f"{k}. {name}")
        print("5. Done")
        choice = input("Enter a number: ").strip()
        if choice == "5":
            break
        elif choice in category_map:
            name, func = category_map[choice]
            emissions[name] = func()
            print(f"{name} updated.")
        else:
            print("Invalid choice.")
    new_total = summarize(emissions)
    show_rank(new_total)

def carbon_challenge():
    tips = [
        "Day 1: Replace 1 car trip with walking or biking.",
        "Day 2: Eat a plant-based meal.",
        "Day 3: Unplug devices not in use.",
        "Day 4: Switch to cold water laundry.",
        "Day 5: Take a 5-minute shower."
    ] + [f"Day {i}: Continue your sustainable actions." for i in range(6, 31)]
    print("\n\U0001F4C6 Your 30-Day Carbon Challenge:")
    for tip in tips:
        print(tip)

def award_badges(emissions, total):
    badges = []
    if emissions["Diet"][1] == "Vegan":
        badges.append("\ud83c\udf66 Plant Power Badge")
    if "miles" in emissions["Transportation"][1] and float(emissions["Transportation"][1].split()[0]) < 1000:
        badges.append("\ud83d\udeb6 Minimal Miles Badge")
    if emissions["Waste"][1] == "Yes" or emissions["Waste"][1] == "Now Recycling":
        badges.append("\u267B\ufe0f Recycler Badge")
    if total < 8:
        badges.append("\ud83c\udf1f Eco Hero Badge")
    if badges:
        print("\n\ud83c\udfc5 You earned the following badges:")
        for badge in badges:
            print(f" - {badge}")

def simulate_improvements(emissions):
    print("\nWould you like to simulate the suggested improvements to see how much better you'd do?")
    choice = input("Proceed with simulation? (y/n): ").strip().lower()
    if choice != 'y':
        return
    apply_suggestions(emissions)
    new_total = summarize(emissions)
    show_rank(new_total)
    award_badges(emissions, new_total)
    if input("Would you like to save your improved results? (y/n): ").strip().lower() == 'y':
        export_summary(emissions, new_total)

def main():
    print("\U0001F33F Welcome to the Carbon Footprint Calculator")
    unit_system = choose_unit_system()
    emissions = {
        "Transportation": transportation(unit_system),
        "Energy": energy(),
        "Diet": diet(),
        "Waste": waste()
    }
    total = summarize(emissions)
    recommend(emissions)
    show_rank(total)
    award_badges(emissions, total)
    if input("Would you like to save your results? (y/n): ").strip().lower() == 'y':
        export_summary(emissions, total)
    if input("Want to try your own manual changes before suggestions? (y/n): ").strip().lower() == 'y':
        test_manual_changes(emissions)
    simulate_improvements(emissions)
    if input("Would you like to see your 30-day carbon challenge? (y/n): ").strip().lower() == 'y':
        carbon_challenge()

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nExited by user.")
        sys.exit(0)


🌿 Welcome to the Carbon Footprint Calculator

--- Transportation ---

--- Home Energy ---

--- Diet ---
Select your diet type:
1. Vegan
2. Vegetarian
3. Pescatarian
4. Omnivore

--- Waste ---
Do you recycle regularly?
1. Yes
2. No

===== Annual Carbon Footprint Summary =====
    Transportation: 4.62 metric tons CO2e (15600 miles/year)
🔺 Energy: 16.56 metric tons CO2e (18000 kWh/year)
    Diet: 1.90 metric tons CO2e (Pescatarian)
    Waste: 0.25 metric tons CO2e (Yes)

TOTAL: 23.33 metric tons CO2e
🌍 Equivalent to driving 58332 miles or powering 5.1 homes/year.
📊 Compared to U.S. Average (16.5 tons): 141.4%
🌎 If everyone lived like you, we’d need 13.7 Earths.

📌 Emission Breakdown (threshold: 25%)
- Energy: 16.56 tons (71.0%) — ⚡ Switch to LED bulbs, unplug devices, or explore renewable energy.

===== Performance Ranking =====
❌ You are above the U.S. average. Try the suggestions to reduce your impact.
📉 You're emitting -6.83 tons less than the average American.

 - ♻️ Recycler Badge


ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/jupyter_client/session.py", line 100, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 30-31: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/zmq/eventloop/zmqstream.py", line 584, in _run_callback
    f = callback(*args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/iostream.py", line 170, in _handle_event
    event_f()
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/iostream.py", line 649, in _flush
    self.session.send(
  File "/opt/conda/lib/python3.11/site-packages/jupyter_client/session.py", line 852, in send
    to_s